# Day 20

In [1]:
from ipynb.fs.defs.utils import read_lines 
from collections import deque 
from collections import defaultdict
from math import lcm

In [2]:
puzzle_input = read_lines('day20.txt')

In [3]:
test_input = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a""".splitlines()

In [4]:
def parse_module(line):
    name, dest_list = line.split(' -> ')
    if name == 'broadcaster':
        type = 'b'
        id = name
    else:
        type = name[0]
        id = name[1:]
    dests = dest_list.split(', ')
    return id, type, dests

In [5]:
def parse_input(inp):
    return [parse_module(l) for l in inp]

In [6]:
test_defs = parse_input(test_input)
test_defs

[('broadcaster', 'b', ['a', 'b', 'c']),
 ('a', '%', ['b']),
 ('b', '%', ['c']),
 ('c', '%', ['inv']),
 ('inv', '&', ['a'])]

In [7]:
def init(module_definitions):
    modules = { id: {'type': type, 'mem': None, 'out': dests } for id, type, dests in module_definitions }
    modules['rx'] = {'type': 'x', 'mem': 0, 'out': [] }
    input_modules = defaultdict(list)
    for id, state in modules.items():
        for o in state['out']:
            input_modules[o].append(id)
    for id, state in modules.items():
        if state['type'] == '%':
            state['mem'] = False
        elif state['type'] == '&':
            state['mem'] = {inp: 0 for inp in input_modules[id]}
    return modules

In [8]:
test_modules = init(test_defs)
test_modules

{'broadcaster': {'type': 'b', 'mem': None, 'out': ['a', 'b', 'c']},
 'a': {'type': '%', 'mem': False, 'out': ['b']},
 'b': {'type': '%', 'mem': False, 'out': ['c']},
 'c': {'type': '%', 'mem': False, 'out': ['inv']},
 'inv': {'type': '&', 'mem': {'c': 0}, 'out': ['a']},
 'rx': {'type': 'x', 'mem': 0, 'out': []}}

In [9]:
def consume(event, modules, i):
    id, pulse, src = event
    #print(f'{src} -- {pulse} --> {id}')
    actions = []
    state = modules[id]
    type = state['type']
    if type == 'b':
        actions = [(o, pulse, id) for o in state['out']]
    elif type == '%':
        if pulse == 0:
            state['mem'] = not state['mem']
            out_p = 1 if state['mem']  else 0
            actions = [(o, out_p, id) for o in state['out']]
    elif type == '&':
        if id == 'll' and  pulse == 1:
            print(f'{src}, {i}')
        state['mem'][src] = pulse
        if not any(v == 0 for v in state['mem'].values()): # all 1
            out_p = 0
        else:
            out_p = 1
        actions = [(o, out_p, id) for o in state['out']]
    return actions, modules

In [10]:
test_modules = init(test_defs)
consume(('inv', 1, 'c'), test_modules, 0)

([('a', 0, 'inv')],
 {'broadcaster': {'type': 'b', 'mem': None, 'out': ['a', 'b', 'c']},
  'a': {'type': '%', 'mem': False, 'out': ['b']},
  'b': {'type': '%', 'mem': False, 'out': ['c']},
  'c': {'type': '%', 'mem': False, 'out': ['inv']},
  'inv': {'type': '&', 'mem': {'c': 1}, 'out': ['a']},
  'rx': {'type': 'x', 'mem': 0, 'out': []}})

In [11]:
def push_button(modules, i):
    actions = deque()
    actions.append(('broadcaster', 0, None))
    l_pulse = 0
    h_pulse = 0
    while len(actions) > 0:
        event = actions.popleft()
        if event[1] == 0:
            l_pulse += 1
        else:
            h_pulse += 1
        new_actions, modules = consume(event, modules, i)
        for act in new_actions:
            actions.append(act)
    return modules, l_pulse, h_pulse

In [12]:
push_button(init(test_defs), 0)

({'broadcaster': {'type': 'b', 'mem': None, 'out': ['a', 'b', 'c']},
  'a': {'type': '%', 'mem': False, 'out': ['b']},
  'b': {'type': '%', 'mem': False, 'out': ['c']},
  'c': {'type': '%', 'mem': False, 'out': ['inv']},
  'inv': {'type': '&', 'mem': {'c': 0}, 'out': ['a']},
  'rx': {'type': 'x', 'mem': 0, 'out': []}},
 8,
 4)

In [13]:
test2_defs = test_defs
modules, l, h = push_button(init(test2_defs), 0)
print(modules, l, h)
print('second push')
modules, l, h = push_button(init(test2_defs), 0)
print(modules, l, h)

{'broadcaster': {'type': 'b', 'mem': None, 'out': ['a', 'b', 'c']}, 'a': {'type': '%', 'mem': False, 'out': ['b']}, 'b': {'type': '%', 'mem': False, 'out': ['c']}, 'c': {'type': '%', 'mem': False, 'out': ['inv']}, 'inv': {'type': '&', 'mem': {'c': 0}, 'out': ['a']}, 'rx': {'type': 'x', 'mem': 0, 'out': []}} 8 4
second push
{'broadcaster': {'type': 'b', 'mem': None, 'out': ['a', 'b', 'c']}, 'a': {'type': '%', 'mem': False, 'out': ['b']}, 'b': {'type': '%', 'mem': False, 'out': ['c']}, 'c': {'type': '%', 'mem': False, 'out': ['inv']}, 'inv': {'type': '&', 'mem': {'c': 0}, 'out': ['a']}, 'rx': {'type': 'x', 'mem': 0, 'out': []}} 8 4


In [14]:
def part1(inp):
    defs = parse_input(inp)
    modules = init(defs)
    l_pulse = 0
    h_pulse = 0
    for _ in range(1000):
        modules, l, h = push_button(modules,0 )
        l_pulse += l
        h_pulse += h
    return l_pulse * h_pulse

In [15]:
part1(puzzle_input)

743090292

In [16]:
def print_graph(modules):
    todo = deque()
    processed = []
    output_lines = []
    todo.append('broadcaster')
    while len(todo) > 0:
        node = todo.popleft()
        if not node in processed:
            output_lines += [f'{node} {o}' for o in modules[node]['out']]
            processed.append(node)
            for o in modules[node]['out']:
                todo.append(o)
    print('\n'.join(output_lines))

In [17]:
print_graph(init(parse_input(puzzle_input)))

broadcaster lp
broadcaster fn
broadcaster tp
broadcaster zz
lp gv
lp tj
fn th
fn bv
tp hb
tp sv
zz fj
zz ff
gv ct
tj xh
tj gv
tj gz
tj bt
tj ct
tj vb
tj lp
th tq
th lr
th vm
th fn
th qr
bv th
bv lr
hb sv
hb xf
hb kv
hb tp
hb mx
sv ch
fj rs
ff kl
ff zd
ff lg
ff zz
ff fj
ff zp
ct xh
xh js
gz dg
bt kc
vb ll
tq mf
lr tq
vm ll
qr jj
xf km
kv ll
mx xp
ch hb
ch mv
rs sq
rs ff
kl ll
zd zp
lg vd
zp bn
js tj
js bt
dg rf
dg tj
kc qm
kc tj
ll rx
mf th
mf qr
jj xt
jj th
km fz
km hb
xp hb
xp xf
mv hb
mv mx
sq zd
sq ff
vd ff
vd mb
bn ff
bn lg
rf lq
rf tj
qm gz
qm tj
xt rk
xt th
fz hb
fz dx
mb ld
mb ff
lq tj
rk rc
rk th
dx hb
dx fx
ld fq
ld ff
rc st
rc th
fx rl
fx hb
fq ff
st th
st jh
rl hb
jh th


In [18]:
def part2(inp):
    defs = parse_input(inp)
    modules = init(defs)
    l_pulse = 0
    h_pulse = 0
    i = 1
    #invocs = defaultdict(list)
    while i < 20000:
        modules, l, h = push_button(modules, i)
        i += 1
    return 

In [19]:
part2(puzzle_input)

vb, 3793
kl, 3917
kv, 4013
vm, 4051
vb, 7586
kl, 7834
kv, 8026
vm, 8102
vb, 11379
kl, 11751
kv, 12039
vm, 12153
vb, 15172
kl, 15668
kv, 16052
vm, 16204
vb, 18965
kl, 19585


In [20]:
7586 - 3793

3793

In [21]:
7834 - 3917

3917

In [22]:
8026 - 4013

4013

In [23]:
8102 - 4051

4051

In [24]:
lcm(3793, 3917, 4013, 4051)

241528184647003